In [1]:
from ngsolve import *
import netgen.geom2d as g2d
from netgen.meshing import Pnt
from ngsolve.la import EigenValues_Preconditioner

In [2]:
geo = g2d.CSG2d()
geo.Add(g2d.Circle(Pnt((0,0)), 1.0))
ngmesh = geo.GenerateMesh(maxh=1.0)
mesh = Mesh(ngmesh)
mesh.ngmesh.SecondOrder()
#mesh.Curve(2)
Draw(mesh)

In [3]:
V = VectorH1(mesh, order=2)
u,v = V.TnT()

ndscal = V.ndof//mesh.dim
nv = mesh.nv

eps = lambda U : Grad(U)+Grad(U).trans

In [4]:
V.SetCouplingType(IntRange(0, V.ndof), COUPLING_TYPE.INTERFACE_DOF)
# for k,v in enumerate(mesh.vertices):
#     V.SetCouplingType(k, WIREBASKET_DOF)
#     V.SetCouplingType(ndscal+k, WIREBASKET_DOF)
V.SetCouplingType(IntRange(0, nv), COUPLING_TYPE.WIREBASKET_DOF)
V.SetCouplingType(IntRange(ndscal, ndscal+nv), COUPLING_TYPE.WIREBASKET_DOF)

In [5]:
a = BilinearForm(V, eliminate_internal=True)
a += InnerProduct(eps(u), eps(v))*dx
a += u*v *dx
c1 = Preconditioner(a, "local")
c2 = Preconditioner(a, "h1amg")
c3 = Preconditioner(a, "bddc")
a.Assemble()

In [6]:
gfu = GridFunction(V)
for k,v in enumerate(mesh.vertices):
    x,y = v.point
    gfu.vec[k] = -y
    gfu.vec[ndscal+k] = x

In [7]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=c1)
print("jacobi")
print(lams[0:3], "...\n", lams[-3:])
print("precond1", max(lams) / min(lams))

jacobi
 0.0195599
 0.0217984
 0.173449
 ...
  2.86818
 2.99115
 3.07675

precond1 157.29914220879493


In [8]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=c2)
print("multigrid")
print(lams[0:3], "...\n", lams[-3:])
print("precond2", max(lams) / min(lams))

multigrid
 0.15098
 0.222325
 0.320574
 ...
  0.97362
 0.989566
 0.999412

precond2 6.619509171094502


In [9]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=c3)
print("bddc")
print(lams[0:3], "...\n", lams[-3:])
print("precond3", max(lams) / min(lams))

bddc
       1
 1.07479
 1.15012
 ...
  5.65501
 5.69174
   6.521

precond3 6.520998533330972


In [10]:
def VertexPatchBlocks(fes):
    mesh = fes.mesh
    blocks = []
    freedofs = fes.FreeDofs()
    for v in mesh.vertices:
        vdofs = set()
        for el in mesh[v].elements:
            vdofs |= set(d for d in fes.GetDofNrs(el)
                            if freedofs[d])
        blocks.append(vdofs)
    return blocks

blocks = VertexPatchBlocks(V)
c4 = a.mat.CreateBlockSmoother(blocks)
lams = EigenValues_Preconditioner(mat=a.mat, pre=c3)
print("blockjac")
print(lams[0:3], "...\n", lams[-3:])
print("precond4", max(lams) / min(lams))

blockjac
       1
 1.08123
 1.15635
 ...
  5.65501
 5.69174
   6.521

precond4 6.5209959395835995
